# Data Collection

Importing needed libraries and reading suplied csv from the Numbers database.

In [1]:
import pandas as pd
import requests
from config import tmdb_api_key

In [2]:
budget_df = pd.read_csv('Data/tn.movie_budgets.csv')

Exploring info in the Nubmers database. This includes mostly budget and revenue information, which is what we are looking for.

All information is saved as a string, which will need to be cleaned later.

In [3]:
budget_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [4]:
budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


Searching TMDB just by name gave us lots of incorrect values, because many movies have the same name. Searching by both title and year gives us a much more accurate result.

In [5]:
budget_df['year'] = budget_df['release_date'].map(lambda x: str(x)[-4:])
budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0,2018
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",1999
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",2005
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,2015


In [6]:
def get_tmdb_info(title, year):
    answer = None
    response = requests.get(f'https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}&year={year}')
    try:
        for movie in response.json()['results']:
            if movie['title'] == title:
                answer = movie
                break
    except KeyError:
        answer = None
    return answer

In [8]:
results = []
for i in budget_df.index:
    title = budget_df.movie[i]
    year = budget_df.year[i]
    print(i, title, year)
    title_result = get_tmdb_info(title, year)
    if title_result:
        results.append(title_result)
tmdb_df = pd.DataFrame(results)

0 Avatar 2009
1 Pirates of the Caribbean: On Stranger Tides 2011
2 Dark Phoenix 2019
3 Avengers: Age of Ultron 2015
4 Star Wars Ep. VIII: The Last Jedi 2017
5 Star Wars Ep. VII: The Force Awakens 2015
6 Avengers: Infinity War 2018
7 Pirates of the Caribbean: At Worldâs End 2007
8 Justice League 2017
9 Spectre 2015
10 The Dark Knight Rises 2012
11 Solo: A Star Wars Story 2018
12 The Lone Ranger 2013
13 John Carter 2012
14 Tangled 2010
15 Spider-Man 3 2007
16 Captain America: Civil War 2016
17 Batman v Superman: Dawn of Justice 2016
18 The Hobbit: An Unexpected Journey 2012
19 Harry Potter and the Half-Blood Prince 2009
20 The Hobbit: The Desolation of Smaug 2013
21 The Hobbit: The Battle of the Five Armies 2014
22 The Fate of the Furious 2017
23 Superman Returns 2006
24 Pirates of the Caribbean: Dead Men Tell No Tales 2017
25 Quantum of Solace 2008
26 The Avengers 2012
27 Pirates of the Caribbean: Dead Manâs Chest 2006
28 Man of Steel 2013
29 The Chronicles of Narnia: Prince Caspian

288 Ghost Rider 2007
289 Rampage 2018
290 Charlie's Angels: Full Throttle 2003
291 Elysium 2013
292 Oblivion 2013
293 Power Rangers 2017
294 Stuart Little 2 2002
295 RoboCop 2014
296 The Chronicles of Riddick 2004
297 Fantastic Four 2015
298 Speed Racer 2008
299 The Island 2005
300 How Do You Know? 2010
301 Knight and Day 2010
302 Venom 2018
303 Star Wars Ep. I: The Phantom Menace 1999
304 Star Wars Ep. III: Revenge of the Sith 2005
305 Star Wars Ep. II: Attack of the Clones 2002
306 Monsters, Inc. 2001
307 The Wolverine 2013
308 Dante's Peak 1997
309 The Huntsman: Winterâs War 2016
310 Windtalkers 2002
311 AstÃ©rix aux Jeux Olympiques 2008
312 Gulliver's Travels 2010
313 Ferdinand 2017
314 Deadpool 2 2018
315 Gravity 2013
316 Night at the Museum 2006
317 Mr. and Mrs. Smith 2005
318 San Andreas 2015
319 The Smurfs 2011
320 Live Free or Die Hard 2007
321 The Divergent Series: Insurgent 2015
322 Ocean's Twelve 2004
323 Tomorrow Never Dies 1997
324 The Patriot 2000
325 300: Rise of an E

596 Pompeii 2014
597 Battlefield Earth: A Saga of the Year 3000 2000
598 Looney Tunes: Back in Action 2003
599 Nine 2009
600 Timeline 2003
601 Oceans 2010
602 Babe: Pig in the City 1998
603 The Postman 1997
604 Red Planet 2000
605 Arthur et les Minimoys 2007
606 The Invasion 2007
607 A Sound of Thunder 2005
608 The Lion King 1994
609 Journey 2: The Mysterious Island 2012
610 Jack and Jill 2011
611 A Beautiful Mind 2001
612 Cloudy with a Chance of Meatballs 2 2013
613 Red Dragon 2002
614 Hidalgo 2004
615 The League of Extraordinary Gentlemen 2003
616 Exorcist: The Beginning 2004
617 AstÃ©rix et ObÃ©lix: Au service de Sa MajestÃ© 2012
618 The Little Prince 2016
619 Renegades 2019
620 The Equalizer 2 2018
621 Despicable Me 2 2013
622 2 Fast 2 Furious 2003
623 Children of Men 2006
624 The Adventures of Rocky & Bullwinkle 2000
625 The Secret Life of Pets 2016
626 Independence Day 1996
627 Dr. Seussâ The Grinch 2018
628 Sing 2016
629 Despicable Me 3 2017
630 The Lost World: Jurassic Park 1

933 Dark Water 2005
934 The Avengers 1998
935 The Black Dahlia 2006
936 EDtv 1999
937 The Spirit 2008
938 The Last Castle 2001
939 Inkheart 2009
940 Supernova 2000
941 Flyboys 2006
942 Winterâs Tale 2014
943 Holy Man 1998
944 Meet Dave 2008
945 The Great Raid 2005
946 Rock Dog 2017
947 Mortdecai 2015
948 In the Name of the King: A Dungeon Siege Tale 2008
949 Beyond Borders 2003
950 Xi You Ji zhi Sun Wu Kong San Da Bai Gu Jing 2016
951 The Ridiculous 6 2015
952 Midway 2019
953 Sherlock Gnomes 2018
954 The Kid Who Would Be King 2019
955 Deadpool 2016
956 American Sniper 2014
957 Goosebumps 2015
958 Rambo III 1988
959 Sabrina 1995
960 Red Tails 2012
961 Just Like Heaven 2005
962 The Internship 2013
963 The Flintstones in Viva Rock Vegas 2000
964 Leatherheads 2008
965 Life 2017
966 Did You Hear About the Morgans? 2009
967 Che, Part 1: The Argentine 2008
968 Sex and the City 2008
969 Resident Evil: Afterlife 2010
970 That's My Boy 2012
971 Devil's Advocate 1997
972 Ghost Rider: Spirit of 

1258 Journey to the Center of the Earth 2008
1259 The Pelican Brief 1993
1260 The Princess Diaries 2: Royal Engagement 2004
1261 The Bucket List 2007
1262 The Client 1994
1263 The Muppets 2011
1264 Patriot Games 1992
1265 Monster-in-Law 2005
1266 Training Day 2001
1267 The Girl on the Train 2016
1268 Galaxy Quest 1999
1269 Scary Movie 2 2001
1270 Blade 1998
1271 Coach Carter 2005
1272 The Bounty Hunter 2010
1273 Changing Lanes 2002
1274 Anaconda 1997
1275 Underworld: Evolution 2006
1276 Coyote Ugly 2000
1277 Love Actually 2003
1278 Jack 1996
1279 The Specialist 1994
1280 Office Christmas Party 2016
1281 Tin Cup 1996
1282 Yours, Mine and Ours 2005
1283 Kicking and Screaming 2005
1284 The Hitchhiker's Guide to the Galaxy 2005
1285 Resident Evil: Extinction 2007
1286 Fat Albert 2004
1287 Savages 2012
1288 The River Wild 1994
1289 Blended 2014
1290 All Eyez on Me 2017
1291 Last Holiday 2006
1292 16 Blocks 2006
1293 The Indian in the Cupboard 1995
1294 Cellular 2004
1295 The Ant Bully 2006


...

4411 Lady in White 1988
4412 Driving Lessons 2006
4413 Without Men 2011
4414 Treading Water 2015
4415 Pat Garrett and Billy the Kid 1973
4416 Camping Sauvage 2004
4417 Barfi 2012
4418 Dear Frankie 2005
4419 The Assassination of Richard Nixon 2004
4420 Le nom des gens 2011
4421 Savage Grace 2008
4422 Out of the Blue 2007
4423 Police Academy 1984
4424 Four Weddings and a Funeral 1994
4425 The Blue Lagoon 1980
4426 Fast Times at Ridgemont High 1982
4427 Secrets & Lies 1996
4428 25th Hour 2002
4429 After Hours 1985
4430 Moby Dick 1956
4431 The Lobster 2016
4432 Bound 1996
4433 Requiem for a Dream 2000
4434 State Fair 1962
4435 LÃ¥t den rÃ¤tte komma in 2008
4436 Tango 1999
4437 Salvador 1986
4438 Donnie Darko 2001
4439 Salvando al Soldado Perez 2011
4440 Karakter 1998
4441 Nannerl la soeur de Mozart 2011
4442 Spun 2003
4443 Life During Wartime 2010
4444 Chinjeolhan geumjassi 2006
4445 Blackthorn 2011
4446 Maggie 2015
4447 Lilja 4-ever 2003
4448 After.Life 2010
4449 Mean Machine 2002
4450 Fo

4733 The D Train 2015
4734 Bachelorette 2012
4735 Fish Tank 2010
4736 Everybody Wants to Be Italian 2008
4737 Creature 2011
4738 The Bounce Back 2016
4739 Freeway 1996
4740 Love's Abiding Joy 2006
4741 Dead Alive 1993
4742 Chain Letter 2010
4743 Tim and Eric's Billion Dollar Movie 2012
4744 Holly 2007
4745 The Eclipse 2010
4746 The Grand 2008
4747 Sommersturm 2006
4748 Fort McCoy 2014
4749 The Gambler 1999
4750 Just Looking 2000
4751 Before We Go 2015
4752 The Divide 2012
4753 Tanner Hall 2011
4754 My Big Fat Independent Movie 2005
4755 Demonic 2017
4756 Ã l\'intÃ©rieur 2008
4757 They Came Together 2014
4758 Open Road 2013
4759 Alice in Wonderland 1951
4760 Barry Munday 2010
4761 The Deported 2009
4762 Chicago Overcoat 2009
4763 Never Back Down 2: The Beatdown 2011
4764 AWOL-72 2015
4765 30 Nights Of Paranormal Activity With The Devil Inside The Girl With The Dragon Tattoo 2013
4766 Enter the Dangerous Mind 2015
4767 Something Wicked 2015
4768 Point Blank 1967
4769 Iguana 2014
4770 Fou

5039 La misma luna 2008
5040 Conquest of the Planet of the Apes 1972
5041 The Lunchbox 2014
5042 Grace Unplugged 2013
5043 Happy, Texas 1999
5044 Saul fia 2015
5045 My Summer of Love 2005
5046 Yes 2005
5047 Braid 2019
5048 From Here to Eternity 1953
5049 You Can't Take It With You 1938
5050 Foolish 1999
5051 She Wore a Yellow Ribbon 1949
5052 The Conversation 1974
5053 N-Secure 2010
5054 Sukkar banat 2008
5055 Out of the Dark 1989
5056 Bubble 2006
5057 Dil Jo Bhi Kahey... 2005
5058 Mississippi Mermaid 1999
5059 Two Night Stand 2014
5060 I Love Your Work 2005
5061 The Kingâs Thief 1955
5062 Insidious 2011
5063 Moonlight 2016
5064 Cabin Fever 2003
5065 Waitress 2007
5066 Kickboxer 1989
5067 Bloodsport 1988
5068 Mr. Smith Goes To Washington 1939
5069 Unsane 2018
5070 Kids 1995
5071 The Squid and the Whale 2005
5072 Kissing Jessica Stein 2002
5073 Spellbound 1945
5074 Dawn of the Dead 1979
5075 Exotica 1994
5076 The To Do List 2013
5077 The Gatekeepers 2013
5078 Buffalo '66 1998
5079 Rep

5352 Harrison Montgomery 2008
5353 Celeste and Jesse Forever 2012
5354 Time Changer 2002
5355 London to Brighton 2009
5356 Crying With Laughter 2009
5357 Billy Jack 1971
5358 Jodaeiye Nader az Simin 2011
5359 Welcome to the Dollhouse 1996
5360 Raising Victor Vargas 2003
5361 Ruby in Paradise 1993
5362 Pandora's Box 2002
5363 Cama adentro 2007
5364 Deterrence 2000
5365 The Mudge Boy 2004
5366 The Young Unknowns 2003
5367 DÃ¸d snÃ¸ 2009
5368 Not Cool 2014
5369 Kill List 2012
5370 Vessel 2015
5371 Saints and Soldiers 2004
5372 American Graffiti 1973
5373 Iraq for Sale: The War Profiteers 2006
5374 Kevin Hart: Laugh at My Pain 2011
5375 Aqua Teen Hunger Force Colon Movie Film for Theaters 2007
5376 Safety Not Guaranteed 2012
5377 The Innkeepers 2012
5378 Il conformista 2014
5379 Interview with the Assassin 2002
5380 Undead 2005
5381 Donkey Punch 2009
5382 All the Boys Love Mandy Lane 2013
5383 Destiny 2014
5384 Bled 2009
5385 Adulterers 2016
5386 La mariÃ©e Ã©tait en noir 1968
5387 High No

5661 Speak No Evil 2013
5662 A Charlie Brown Christmas 2009
5663 Aroused 2013
5664 Escaping the Holocost 2015
5665 The Texas Chainsaw Massacre 1974
5666 Roger & Me 1989
5667 Sound of My Voice 2012
5668 Cat People 1942
5669 An American in Hollywood 2014
5670 The Brain That Sings 2015
5671 Romantic Schemer 2015
5672 Your Sister's Sister 2012
5673 The Blood of My Brother: A Story of Death in Iraq 2006
5674 A Dog's Breakfast 2007
5675 Une Femme MariÃ©e 2016
5676 Night of the Living Dead 1968
5677 The Birth of a Nation 1915
5678 The Work and the Story 2003
5679 The Gallows 2015
5680 Facing the Giants 2006
5681 Eraserhead 1977
5682 Hollywood Shuffle 1987
5683 Over the Hill to the Poorhouse 1920
5684 The Mighty 1998
5685 A Ghost Story 2017
5686 Penitentiary 1980
5687 The Lost Skeleton of Cadavra 2004
5688 Cheap Thrills 2014
5689 The Past is a Grotesque Animal 2014
5690 Keeping it Real: The Adventures of Greg Walloch 2001
5691 Indie Game: The Movie 2012
5692 Dude, Where's My Dog 2014
5693 Echo

The TMDB info looks useable. 4800 matches from the 5800 from the original dataset.

In [9]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4775 entries, 0 to 4774
Data columns (total 14 columns):
popularity           4775 non-null float64
vote_count           4775 non-null int64
video                4775 non-null bool
poster_path          4729 non-null object
id                   4775 non-null int64
adult                4775 non-null bool
backdrop_path        4458 non-null object
original_language    4775 non-null object
original_title       4775 non-null object
genre_ids            4775 non-null object
title                4775 non-null object
vote_average         4775 non-null float64
overview             4775 non-null object
release_date         4775 non-null object
dtypes: bool(2), float64(2), int64(2), object(8)
memory usage: 457.1+ KB


In [10]:
tmdb_df

,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,82.100,22200,False,/6EiRUJpuoeQPghrs3YNktfnqOVh.jpg,19995,False,/bIL7ENqh1egWTxN41sM2W42JqPc.jpg,en,Avatar,"[28, 12, 14, 878]",Avatar,7.5,"In the 22nd century, a paraplegic Marine is di...",2009-12-10
1,44.410,10319,False,/9zxsSzXx2UO3yNoS4j97vs4uOhc.jpg,1865,False,/bfTKq6mGHt2CzZlBU31JHF9bMog.jpg,en,Pirates of the Caribbean: On Stranger Tides,"[28, 12, 14]",Pirates of the Caribbean: On Stranger Tides,6.5,Captain Jack Sparrow crosses paths with a woma...,2011-05-14
2,115.004,4000,False,/cCTJPelKGLhALq3r51A9uMonxKj.jpg,320288,False,/phxiKFDvPeQj4AbkvJLmuZEieDU.jpg,en,Dark Phoenix,"[28, 12, 878]",Dark Phoenix,6.1,The X-Men face their most formidable and power...,2019-06-05
3,79.895,16666,False,/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg,99861,False,/8i6ZDk97Vyh0jHohMG4vFeFuKJh.jpg,en,Avengers: Age of Ultron,"[28, 12, 878]",Avengers: Age of Ultron,7.3,When Tony Stark tries to jumpstart a dormant p...,2015-04-22
4,204.940,19949,False,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,299536,False,/lmZFxXgJE3vgrciwuDib0N8CfQo.jpg,en,Avengers: Infinity War,"[28, 12, 878]",Avengers: Infinity War,8.3,As the Avengers and their allies have continue...,2018-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4770,0.600,0,False,/kOk47vD8E0dScAyNVNCXhsncaE4.jpg,586483,False,None,en,Clean,[],Clean,0.0,A dangerous serial killer decides to go out wi...,2006-03-05
4771,0.600,0,False,/2YkIO5qjfhyICn5wXhLYMJLk152.jpg,579116,False,None,en,The Rise and Fall of Miss Thang,[],The Rise and Fall of Miss Thang,0.0,An irresponsible party girl begins a journey t...,2008-08-14
4772,14.113,647,False,/swPcAhFNbREyvBaKeicVkF8WU5B.jpg,9367,False,/3oSXZcnd8ALYNjcy2v8t5j0THgE.jpg,es,El Mariachi,"[28, 80, 53]",El Mariachi,6.7,El Mariachi just wants to play his guitar and ...,1992-09-04
4773,50.245,1382,False,/xEoq2WmDzpzxhkHEsmOYOg6BPg6.jpg,14337,False,/gIFbfh9O4HBrO6numxVBxtZ9fTL.jpg,en,Primer,"[18, 878, 53]",Primer,6.9,Friends and fledgling entrepreneurs invent a d...,2004-10-08


Saving the TMDB info to a csv file. This will be merged with the budget data later, and cleaned.

In [11]:
tmdb_df.to_csv('Data/tmdb_df.csv', index = False)